In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import display
import warnings

warnings.filterwarnings('ignore', category=UserWarning)
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

In [2]:
processed_seoul = pd.read_csv('processed_seoul.csv')
processed_gyeonggi = pd.read_csv('processed_gyeonggi.csv')
processed_incheon = pd.read_csv('processed_incheon.csv')
processed_busan = pd.read_csv('processed_busan.csv')
processed_daegu = pd.read_csv('processed_daegu.csv')
processed_gwangju = pd.read_csv('processed_gwangju.csv')
processed_daejeon = pd.read_csv('processed_daejeon.csv')
processed_ulsan = pd.read_csv('processed_ulsan.csv')

In [3]:
# 도시별 df 딕셔너리
city_dfs = {
    '서울특별시': processed_seoul,
    '경기도': processed_gyeonggi,
    '인천광역시': processed_incheon,
    '부산광역시': processed_busan,
    '대구광역시': processed_daegu,
    '광주광역시': processed_gwangju,
    '대전광역시': processed_daejeon,
    '울산광역시': processed_ulsan
}

In [4]:
"""
데이터셋 칼럼 설명

'권역': 행정 구역 기준 권역 정보 (수도권/지방)
'시군구': 시/군/구 단위 행정 구역명
'번지': 도로명 주소의 번지 정보
'본번': 도로명 주소의 본번
'부번': 도로명 주소의 부번
'단지명': 아파트 단지 이름
'계약년월': 계약 발생 연월 (YYYYMM)
'계약년도': 계약 발생 연도 (YYYY)
'계약월': 계약 발생 월 (1~12)
'계약일': 계약 발생 일 (1~31)
'contract_season': 계약 발생 계절 (봄/여름/가을/겨울)
'area_bin': 전용면적 기준 면적 구간
    - 분류 기준:
        - ≤60 ㎡ : 소형
        - 61~85 ㎡ : 중소형
        - 86~135 ㎡ : 중대형
        - >135 ㎡ : 대형
'전용면적(㎡)': 아파트의 전용면적 (㎡)
'거래금액(만원)': 실제 거래 금액 (만원)
'price_per_m2': 평당 가격 (만원/㎡)
'층': 거래된 주택의 층수 (int64)
'건축년도': 건물 준공 연도 (int64)
'building_age': 건물 연식 (계약년도 - 건축년도)
'is_new_building': 신축 여부 (True: 신축)
"""

processed_seoul.head()

,권역,시군구,번지,본번,부번,단지명,계약년월,계약년도,계약월,계약일,contract_season,area_bin,전용면적(㎡),거래금액(만원),price_per_m2,층,건축년도,building_age,is_new_building
0,수도권,서울특별시 강남구 개포동,1164-12,1164,12,새롬(1164-12),201205,2012,5,17,봄,중소형,73.50,38000.0,517.0,6,2000,12,False
1,수도권,서울특별시 강남구 개포동,1164-12,1164,12,새롬(1164-12),201507,2015,7,28,여름,중소형,73.50,42000.0,571.4,3,2000,15,False
2,수도권,서울특별시 강남구 개포동,1164-12,1164,12,새롬(1164-12),201709,2017,9,23,가을,중소형,71.72,64500.0,899.3,7,2000,17,False
3,수도권,서울특별시 강남구 개포동,1164-12,1164,12,새롬(1164-12),201707,2017,7,21,여름,중소형,73.50,55400.0,753.7,2,2000,17,False
4,수도권,서울특별시 강남구 개포동,1164-13,1164,13,새롬(1164-13),201001,2010,1,5,겨울,소형,59.67,35000.0,586.6,3,2000,10,False


### 데이터 시각적 품질 진단

In [ ]:
numeric_vars = ['전용면적(㎡)', '거래금액(만원)', 'price_per_m2', '계약년도', '건축년도', 'building_age']
categorical_vars = ['area_bin', 'contract_season', 'is_new_building']

all_data = pd.DataFrame()

for city_name, df in city_dfs.items():
    print(f"=== {city_name.upper()} 주요 변수 분석 ===\n")
    
    # 1. 수치형 변수 히스토그램 + KDE
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    axes = axes.ravel()
    for i, var in enumerate(numeric_vars):
        ax = axes[i]
        bins = df[var].nunique() if var in ['계약년도', '건축년도'] else 30
        sns.histplot(df[var], bins=bins, kde=True, color='skyblue', edgecolor='black', ax=ax)
        ax.set_title(f'{var} 분포\n(평균: {df[var].mean():.2f}, 왜도: {df[var].skew():.2f})')
        ax.axvline(df[var].mean(), color='red', linestyle='--', alpha=0.8, label='평균')
        ax.legend(labels=['커널밀도', '평균'])
    plt.tight_layout()
    plt.show()
    
    # 2. 카테고리 변수 분포 (Matplotlib 사용)
    fig, axes = plt.subplots(1, len(categorical_vars), figsize=(18, 5))
    if len(categorical_vars) == 1:
        axes = [axes]
    for i, cat_var in enumerate(categorical_vars):
        counts = df[cat_var].value_counts()
        x = np.arange(len(counts))
        # 색상 리스트: 카테고리 개수에 맞춰 지정
        colors = plt.cm.tab10.colors[:len(counts)]
        axes[i].bar(x, counts.values, color=colors, edgecolor='black')
        axes[i].set_xticks(x)
        axes[i].set_xticklabels(counts.index)
        axes[i].set_title(f'{cat_var} 분포')
        axes[i].set_xlabel(cat_var)
        axes[i].set_ylabel('빈도')
        for xi, val in zip(x, counts.values):
            axes[i].text(xi, val, str(val), ha='center', va='bottom', fontsize=9)
    plt.suptitle(f'{city_name.upper()} 카테고리 변수 분포', fontsize=16)
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()
    
    all_data = pd.concat([all_data, df], ignore_index=True)

# 3. 전체 도시 합친 데이터 시각화
print("=== 전체 도시 데이터 시각화 ===\n")

# 수치형 변수 히스토그램 + KDE
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.ravel()
for i, var in enumerate(numeric_vars):
    ax = axes[i]
    bins = all_data[var].nunique() if var in ['계약년도', '건축년도'] else 30
    sns.histplot(all_data[var], bins=bins, kde=True, color='lightgreen', edgecolor='black', ax=ax)
    ax.set_title(f'{var} 분포 (전체 도시)\n(평균: {all_data[var].mean():.2f}, 왜도: {all_data[var].skew():.2f})')
    ax.set_xlabel(var)
    ax.set_ylabel('빈도')
    ax.axvline(all_data[var].mean(), color='red', linestyle='--', alpha=0.8, label='평균')
    ax.legend(labels=['커널밀도', '평균'])
plt.tight_layout()
plt.show()

# 카테고리 변수 분포 (전체 도시)
fig, axes = plt.subplots(1, len(categorical_vars), figsize=(18, 5))
if len(categorical_vars) == 1:
    axes = [axes]
for i, cat_var in enumerate(categorical_vars):
    counts = all_data[cat_var].value_counts()
    x = np.arange(len(counts))
    colors = plt.cm.tab10.colors[:len(counts)]
    axes[i].bar(x, counts.values, color=colors, edgecolor='black')
    axes[i].set_xticks(x)
    axes[i].set_xticklabels(counts.index)
    axes[i].set_title(f'{cat_var} 분포 (전체 도시)')
    axes[i].set_xlabel(cat_var)
    axes[i].set_ylabel('빈도')
    for xi, val in zip(x, counts.values):
        axes[i].text(xi, val, str(val), ha='center', va='bottom', fontsize=9)
plt.suptitle('전체 도시 카테고리 변수 분포', fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

### 수치형 변수 상관관계 분석

In [ ]:
from sklearn.feature_selection import mutual_info_regression

# 제외할 컬럼
exclude_cols = ['본번', '부번', '계약년월', '계약일']

# 결과를 저장할 딕셔너리
final_results = {
    '거래금액(만원)': {},
    'price_per_m2': {}
}

# 도시별 상관관계 분석 및 시각화
for city, df in city_dfs.items():
    print(f"\n\n========== {city} ==========")
    # 수치형 변수만 선택 (제외 컬럼 제거)
    numeric_df = df.select_dtypes(include=['int64', 'float64']).drop(columns=exclude_cols, errors='ignore').dropna()
    
    # Mutual Information (MI) 분석
    X = numeric_df.drop(columns=['거래금액(만원)', 'price_per_m2'], errors='ignore')
    y_targets = {
        '거래금액(만원)': numeric_df['거래금액(만원)'],
        'price_per_m2': numeric_df['price_per_m2']
    }
    
    mi_results = {}  # MI 결과 저장용
    print("\n--- 상호 정보량 (Mutual Information) ---")
    for target_name, y in y_targets.items():
        mi = mutual_info_regression(X, y, random_state=42)
        mi_series = pd.Series(mi, index=X.columns).sort_values(ascending=False)
        # 0.1 이상만 출력
        filtered_mi = mi_series[mi_series >= 0.1]
        mi_results[target_name] = mi_series  # 전체 MI 값 저장
        print(f"\n[{target_name}] (MI>=0.1)")
        print(filtered_mi if not filtered_mi.empty else "조건 만족 없음")
    
    # MI 시각화
    fig, axes = plt.subplots(1, 2, figsize=(20, 6))
    for idx, (target_name, mi_series) in enumerate(mi_results.items()):
        sns.barplot(x=mi_series.index, y=mi_series.values, color='skyblue', ax=axes[idx])
        axes[idx].axhline(0.1, color='red', linestyle='--', linewidth=2, label='커트라인: 0.1')
        axes[idx].set_xticklabels(axes[idx].get_xticklabels(), rotation=45, ha='right')
        axes[idx].set_title(f"{city} - Mutual Information ({target_name})", fontsize=14)
        axes[idx].set_ylabel("MI 값")
        axes[idx].set_xlabel("변수")
        axes[idx].legend()
    plt.tight_layout()
    plt.show()
    
    # 피어슨 & 스피어만 상관계수 (타겟 기준만)
    pearson_corr = numeric_df.corr(method='pearson').loc[['거래금액(만원)', 'price_per_m2'], :]
    spearman_corr = numeric_df.corr(method='spearman').loc[['거래금액(만원)', 'price_per_m2'], :]
    
    # 피어슨 & 스피어만 상관계수 시각화
    fig, axes = plt.subplots(1, 2, figsize=(20, 8))
    a1, a2 = axes[0], axes[1]
    
    # 피어슨 상관관계
    sns.heatmap(pearson_corr, annot=True, fmt='.2f', cmap='RdBu_r', center=0,
                square=True, ax=a1, cbar_kws={'label': 'Pearson 상관계수'})
    a1.set_title(f'{city} - 피어슨 상관관계 (타겟 기준)', fontsize=14)
    
    # 스피어만 상관관계
    sns.heatmap(spearman_corr, annot=True, fmt='.2f', cmap='RdBu_r', center=0,
                square=True, ax=a2, cbar_kws={'label': 'Spearman 상관계수'})
    a2.set_title(f'{city} - 스피어만 상관관계 (타겟 기준)', fontsize=14)
    
    plt.tight_layout()
    plt.show()
    
    # MI >= 0.1 이면서 |r| < 0.2 인 컬럼 찾기
    for target_name, mi_series in mi_results.items():
        candidates = []
        for col, mi_val in mi_series.items():
            if mi_val >= 0.1:
                pearson_r = pearson_corr.loc[target_name, col] if col in pearson_corr.columns else None
                spearman_r = spearman_corr.loc[target_name, col] if col in spearman_corr.columns else None
                # Pearson, Spearman abs(r) < 0.2 조건 확인
                if pearson_r is not None and spearman_r is not None:
                    if abs(pearson_r) < 0.2 or abs(spearman_r) < 0.2:
                        candidates.append(col)
        final_results[target_name][city] = candidates if candidates else "❌ 조건 만족 없음"

# 최종 결과 출력
print("\n\n========== 최종 결과: MI는 높지만 상관관계는 낮은 변수 ==========")
for target_name in final_results:
    print(f"\n[{target_name}]")
    for city, result in final_results[target_name].items():
        print(f"{city}: {result}")

### 카테고리형 변수 상관관계 분석

In [7]:

city_dfs = {
    '서울특별시': processed_seoul,
    '경기도': processed_gyeonggi,
    '인천광역시': processed_incheon,
    '부산광역시': processed_busan,
    '대구광역시': processed_daegu,
    '광주광역시': processed_gwangju,
    '대전광역시': processed_daejeon,
    '울산광역시': processed_ulsan
}


import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_selection import mutual_info_regression
import base64
from io import BytesIO
import matplotlib

# Matplotlib 백엔드를 'Agg'로 설정 (비대화형 환경 지원)
matplotlib.use('Agg')

# 제외할 컬럼
exclude_cols = ['본번', '부번', '계약년월', '계약일']

# HTML 템플릿 시작
html_content = """
<!DOCTYPE html>
<html lang="ko">
<head>
    <meta charset="UTF-8">
    <title>도시별 부동산 데이터 분석 결과</title>
    <style>
        body { font-family: Arial, sans-serif; margin: 20px; }
        h1 { text-align: center; }
        h2 { color: #2c3e50; }
        h3 { color: #34495e; }
        pre { background-color: #f8f9fa; padding: 10px; border-radius: 5px; }
        img { max-width: 100%; height: auto; margin: 10px 0; }
        .section { margin-bottom: 40px; }
        .city-section { border-bottom: 1px solid #ccc; padding-bottom: 20px; }
    </style>
</head>
<body>
    <h1>도시별 부동산 데이터 분석 결과</h1>
"""

# 이미지를 base64로 인코딩하는 함수
def fig_to_base64(fig):
    buf = BytesIO()
    fig.savefig(buf, format='png', bbox_inches='tight')
    buf.seek(0)
    img_base64 = base64.b64encode(buf.getvalue()).decode('utf-8')
    buf.close()
    return img_base64

# 결과를 저장할 딕셔너리
final_results = {
    '거래금액(만원)': {},
    'price_per_m2': {}
}

# 도시별 상관관계 분석 및 HTML 콘텐츠 생성
for city, df in city_dfs.items():
    html_content += f'<div class="city-section"><h2>{city}</h2>\n'
    html_content += '<h3>상호 정보량 (Mutual Information)</h3>\n<pre>\n'

    # 수치형 변수만 선택 (제외 컬럼 제거)
    numeric_df = df.select_dtypes(include=['int64', 'float64']).drop(columns=exclude_cols, errors='ignore').dropna()
    
    # Mutual Information (MI) 분석
    X = numeric_df.drop(columns=['거래금액(만원)', 'price_per_m2'], errors='ignore')
    y_targets = {
        '거래금액(만원)': numeric_df['거래금액(만원)'],
        'price_per_m2': numeric_df['price_per_m2']
    }
    
    mi_results = {}  # MI 결과 저장용
    for target_name, y in y_targets.items():
        mi = mutual_info_regression(X, y, random_state=42)
        mi_series = pd.Series(mi, index=X.columns).sort_values(ascending=False)
        # 0.1 이상만 출력
        filtered_mi = mi_series[mi_series >= 0.1]
        mi_results[target_name] = mi_series  # 전체 MI 값 저장
        html_content += f"[{target_name}] (MI>=0.1)\n"
        html_content += f"{filtered_mi if not filtered_mi.empty else '조건 만족 없음'}\n\n"
    
    html_content += '</pre>\n'
    
    # MI 시각화
    fig, axes = plt.subplots(1, 2, figsize=(20, 6))
    for idx, (target_name, mi_series) in enumerate(mi_results.items()):
        sns.barplot(x=mi_series.index, y=mi_series.values, color='skyblue', ax=axes[idx])
        axes[idx].axhline(0.1, color='red', linestyle='--', linewidth=2, label='커트라인: 0.1')
        axes[idx].set_xticklabels(axes[idx].get_xticklabels(), rotation=45, ha='right')
        axes[idx].set_title(f"{city} - Mutual Information ({target_name})", fontsize=14)
        axes[idx].set_ylabel("MI 값")
        axes[idx].set_xlabel("변수")
        axes[idx].legend()
    plt.tight_layout()
    # 이미지를 base64로 변환하여 HTML에 추가
    img_base64 = fig_to_base64(fig)
    html_content += f'<img src="data:image/png;base64,{img_base64}" alt="{city} MI Plot">\n'
    plt.close(fig)
    
    # 피어슨 & 스피어만 상관계수 (타겟 기준만)
    pearson_corr = numeric_df.corr(method='pearson').loc[['거래금액(만원)', 'price_per_m2'], :]
    spearman_corr = numeric_df.corr(method='spearman').loc[['거래금액(만원)', 'price_per_m2'], :]
    
    # 피어슨 & 스피어만 상관계수 시각화
    fig, axes = plt.subplots(1, 2, figsize=(20, 8))
    a1, a2 = axes[0], axes[1]
    
    # 피어슨 상관관계
    sns.heatmap(pearson_corr, annot=True, fmt='.2f', cmap='RdBu_r', center=0,
                square=True, ax=a1, cbar_kws={'label': 'Pearson 상관계수'})
    a1.set_title(f'{city} - 피어슨 상관관계 (타겟 기준)', fontsize=14)
    
    # 스피어만 상관관계
    sns.heatmap(spearman_corr, annot=True, fmt='.2f', cmap='RdBu_r', center=0,
                square=True, ax=a2, cbar_kws={'label': 'Spearman 상관계수'})
    a2.set_title(f'{city} - 스피어만 상관관계 (타겟 기준)', fontsize=14)
    
    plt.tight_layout()
    # 이미지를 base64로 변환하여 HTML에 추가
    img_base64 = fig_to_base64(fig)
    html_content += f'<img src="data:image/png;base64,{img_base64}" alt="{city} Correlation Heatmap">\n'
    plt.close(fig)
    
    # MI >= 0.1 이면서 |r| < 0.2 인 컬럼 찾기
    for target_name, mi_series in mi_results.items():
        candidates = []
        for col, mi_val in mi_series.items():
            if mi_val >= 0.1:
                pearson_r = pearson_corr.loc[target_name, col] if col in pearson_corr.columns else None
                spearman_r = spearman_corr.loc[target_name, col] if col in spearman_corr.columns else None
                # Pearson, Spearman abs(r) < 0.2 조건 확인
                if pearson_r is not None and spearman_r is not None:
                    if abs(pearson_r) < 0.2 or abs(spearman_r) < 0.2:
                        candidates.append(col)
        final_results[target_name][city] = candidates if candidates else "❌ 조건 만족 없음"
    
    html_content += '</div>\n'

# 최종 결과 출력
html_content += '<div class="section"><h2>최종 결과: MI는 높지만 상관관계는 낮은 변수</h2>\n<pre>\n'
for target_name in final_results:
    html_content += f"[{target_name}]\n"
    for city, result in final_results[target_name].items():
        html_content += f"{city}: {result}\n"
html_content += '</pre>\n</div>\n'

# HTML 템플릿 마무리
html_content += """
</body>
</html>
"""

# HTML 파일로 저장
with open('city_analysis_results.html', 'w', encoding='utf-8') as f:
    f.write(html_content)

print("분석 결과가 'city_analysis_results.html' 파일로 저장되었습니다.")

분석 결과가 'city_analysis_results.html' 파일로 저장되었습니다.
